In [1]:
import torch
import segmentation_models_pytorch as smp

# Returns a Linkenet model which is basically just torch.nn.Module
pan = smp.pan.model.PAN(encoder_name="resnet34",
                       encoder_weights="imagenet",
                       activation="sigmoid",
                       in_channels=3)

# preprocessing input
preprocess = smp.encoders.get_preprocessing_fn('resnet34', pretrained='imagenet')
params = sum(p.numel() for p in pan.parameters())
print("Parameters:", params)

Parameters: 21475816


In [9]:
#Converting torch to onnx

# Onnx input 

x = torch.randn(2,3, 128, 128)
torch_out = pan(x)
torch.onnx.export(pan,
                 x,
                 "pan.onnx",
                 input_names=["input"],
                 output_names=["output"],
                 opset_version=11)

In [10]:
#converting from onnx to openvino 
!python3 /opt/intel/openvino_2021.2.200/deployment_tools/model_optimizer/mo.py --input_model pan.onnx

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/aakash/box_of_ai_tools/Semantic_segmentation/PAN/pan.onnx
	- Path for generated IR: 	/home/aakash/box_of_ai_tools/Semantic_segmentation/PAN/.
	- IR output name: 	pan
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	False
ONNX specific parameters:
Model Optimizer version: 	2021.2.0-1877-176bdf51370-releases/2021/2

[ SUCCESS ] Generated IR version 10 model.
[ SUCCESS ] XML file: /home/aakash/box_of_ai_tools/Semantic_segmentation/PAN/./pan.xml
[ SUCCES

In [11]:
from openvino.inference_engine import IECore
ie = IECore()

# These files including pan.bin, pan.mapping, pan.xml are
# create after converting the onnx model to openvino through the above step
openvino_pan = ie.read_network(model="pan.xml", weights="pan.bin")
exec_pan = ie.load_network(network=openvino_pan, device_name="CPU", num_requests=1)
openvino_out = exec_pan.infer(inputs={"input": x})

In [14]:
print('\x1b[6;30;42m' + 'Torch output:' + '\x1b[0m', torch_out.detach().numpy())
print('\x1b[6;30;42m' + 'Openvino output:' + '\x1b[0m', openvino_out["output"])

Torch output: [[[[0.43092817 0.30854422 0.20820114 ... 0.18358259 0.21274322
    0.2451452 ]
   [0.5305932  0.38945615 0.2646918  ... 0.15158997 0.1732054
    0.19718687]
   [0.6278764  0.4769505  0.33011952 ... 0.12432365 0.13971159
    0.15666354]
   ...
   [0.66548103 0.54779196 0.4245041  ... 0.16052847 0.19457799
    0.23383829]
   [0.67905825 0.5350995  0.38504502 ... 0.16264279 0.1919984
    0.2252276 ]
   [0.6923392  0.5223613  0.34704167 ... 0.16477953 0.18944497
    0.21684417]]]


 [[[0.244274   0.19497882 0.1536095  ... 0.12246855 0.15552442
    0.19551492]
   [0.38328755 0.29324925 0.2169223  ... 0.11468107 0.13296925
    0.15366781]
   [0.5444209  0.41548586 0.2971684  ... 0.1073282  0.11324654
    0.1194476 ]
   ...
   [0.6369233  0.5789023  0.5186171  ... 0.29305935 0.25227395
    0.21543467]
   [0.7050676  0.6365815  0.5620699  ... 0.31708944 0.26066735
    0.2111799 ]
   [0.76513875 0.6905832  0.60459125 ... 0.3421365  0.2692395
    0.20698695]]]]
Onnx output: [[[[0.2